In [22]:
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')

In [2]:
raw_data = pd.read_csv("train.csv")
raw_data.head()

,userID,movieID,rating
0,4490,2109,4
1,5839,3471,4
2,5382,150,3
3,1262,1237,5
4,6005,2273,4


In [3]:
pivot_uu = raw_data.pivot(index="userID", columns= "movieID", values="rating")
pivot_uu.head()

movieID,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
userID,,,,,,,,,,,,,,,,,,,,,
1,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
all_mean = pivot_uu.mean().mean()

In [4]:
pivot_uu.shape

(6040, 3705)

In [20]:
scaled_df = pivot_uu - pivot_uu.mean(axis=1).values.reshape(-1, 1)
scaled_df.fillna(0, inplace=True)
scaled_df.head()

movieID,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
userID,,,,,,,,,,,,,,,,,,,,,
1,-0.571429,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.000000,0.0,0.0,0.0,0.0,-0.299312,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
test_no_labels = pd.read_csv("test_without_labels.csv")
test_no_labels.head()

,IDs
0,629_2683
1,601_904
2,5673_3717
3,5616_1721
4,1099_3697


In [24]:
test_no_labels["userID"] = test_no_labels.IDs.str.split('_').str[0].astype(int)
test_no_labels["movieID"] = test_no_labels.IDs.str.split('_').str[1].astype(int)
test_no_labels.head()

,IDs,userID,movieID
0,629_2683,629,2683
1,601_904,601,904
2,5673_3717,5673,3717
3,5616_1721,5616,1721
4,1099_3697,1099,3697


In [25]:
test_uu = test_no_labels.drop("IDs",axis=1)

In [29]:
pred_ratings_uu = []

for userId,movie in test_uu.itertuples(index=False):
    k = 50
    rated_indices = pivot_uu[np.isnan(pivot_uu[movie]) == False].index
    filtered_df = scaled_df.loc[rated_indices, :]
    if len(filtered_df) == 0:
        pred = all_mean
    else:
        if len(filtered_df) < k:
            k = len(filtered_df)
        
        nn = NearestNeighbors(n_neighbors=k, algorithm="kd_tree")
        nn.fit(filtered_df)
        test_user = scaled_df.loc[[userId], :]
        close_indices = nn.kneighbors(test_user ,return_distance=False)[0]
        close_friends = filtered_df.iloc[close_indices, :]
        pred = pivot_uu.loc[userId, :].mean() + close_friends[movie].mean()
        pred_ratings_uu.append(pred)

In [30]:
len(pred_ratings_uu)

9999

In [31]:
for i in range(len(pred_ratings_uu)): 
    if pred_ratings_uu[i] > 5:
        pred_ratings_uu[i] = 5
    elif pred_ratings_uu[i] < 1:
        pred_ratings_uu[i] = 1

In [32]:
test_no_labels["rating"] = pred_ratings_uu
test_no_labels.drop(columns=["userID","movieID"],axis=1,inplace=True)
test_no_labels.to_csv("result_uu_2804.csv", index=False)